In [1]:
import polars as pl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestRegressor
from snapml import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re

In [17]:


# Load the dataset

# Function to convert entity_value to a common unit (e.g., meters)
def convert_to_meters(value):
    match = re.match(r"([0-9.]+)\s*(\w+)", value)
    if not match:
        return None
    num, unit = match.groups()
    num = float(num)
    unit = unit.lower()
    if unit in ['mm', 'millimeter', 'millimeters', 'millimetre', 'millimetres']:
        return num / 1000
    elif unit in ['cm', 'centimeter', 'centimeters', 'centimetre']:
        return num / 100
    elif unit in ['m', 'meter', 'meters', 'metre', 'metres']:
        return num
    elif unit in ['km', 'kilometer', 'kilometers', 'kilometre', 'kilometres']:
        return num * 1000
    else:
        return None
df = pd.read_csv('../dataset/train.csv')
df = df[df['entity_name'].isin(['width', 'depth', 'height'])]

# Apply the conversion function to the entity_value column
df['entity_value_meters'] = df['entity_value'].apply(convert_to_meters)

# Drop rows with invalid entity_value
df = df.dropna(subset=['entity_value_meters'])

# Encode categorical variables
label_encoder_group = LabelEncoder()
label_encoder_entity = LabelEncoder()

df['group_id_encoded'] = label_encoder_group.fit_transform(df['group_id'])
df['entity_name_encoded'] = label_encoder_entity.fit_transform(df['entity_name'])

# Split the data into training and testing sets
X = df[['group_id_encoded', 'entity_value_meters']]
y = df['entity_name_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Convert to NumPy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
# from sklearn.ensemble import RandomForestClassifier
from snapml import RandomForestClassifier


In [21]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=3, n_jobs=2, verbose=2)
grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_model = grid_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')



Fitting 3 folds for each of 36 candidates, totalling 108 fits


[CV] END max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   6.6s
[CV] END max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   6.7s
[CV] END max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   6.8s
[CV] END max_depth=None, min_samples_leaf=1, n_estimators=200; total time=  14.0s


KeyboardInterrupt: 

In [69]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder_entity.classes_, labels=model.classes_)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.626727842955047
Classification Report:
              precision    recall  f1-score   support

       depth       0.65      0.71      0.68      5910
      height       0.64      0.66      0.65      5681
       width       0.58      0.50      0.54      5627

    accuracy                           0.63     17218
   macro avg       0.47      0.47      0.47     17218
weighted avg       0.62      0.63      0.62     17218



/home/devastatingrpg/miniconda3/envs/dev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2642: UserWarning: labels size, 4, does not match size of target_names, 3
  warnings.warn(
/home/devastatingrpg/miniconda3/envs/dev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/devastatingrpg/miniconda3/envs/dev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/devastatingrpg/miniconda3/envs/dev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMe

In [ ]:
import joblib

# Save the best model to a file
joblib.dump(best_model, 'best_model.pkl')

# Load the model from the file (example usage)
loaded_model = joblib.load('best_model.pkl')

In [ ]:
# # Function to predict dimension type
# def predict_dimension_type(model, group_id, entity_value):
#     group_id_encoded = label_encoder_group.transform([group_id])[0]
#     entity_value_meters = convert_to_meters(entity_value)
#     if entity_value_meters is None:
#         raise ValueError("Invalid entity_value format")
#     input_data = [[group_id_encoded, entity_value_meters]]
#     prediction_encoded = model.predict(input_data)[0]
#     prediction = label_encoder_entity.inverse_transform([prediction_encoded])[0]
#     return prediction

# # Example usage
# group_id = 'some_group_id'
# entity_value = '200 cm'
# predicted_dimension_type = predict_dimension_type(best_model, group_id, entity_value)
# print(f'Predicted Dimension Type: {predicted_dimension_type}')